In [ ]:
import mysql.connector
import sys

# ----------------------------
# Connect to MySQL
# ----------------------------
try:
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Sneha@96",
        database="FINAL_URP"
    )
    cursor = conn.cursor()
    print("Connected to MySQL database successfully!\n", flush=True)
except mysql.connector.Error as err:
    print(f"Error: {err}")
    sys.exit()


# ------------------------------------------------------
# 1. Atomicity Example
# ------------------------------------------------------
try:
    print("=== Atomicity Example ===")

    # Temporary transaction
    # (starts automatically)
    cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
    row = cursor.fetchone()
    if row:
        print("Before update:", row[0], flush=True)

    cursor.execute(
        "UPDATE Paper SET title='Atomic Demo Temporary' WHERE paper_id=1;")
    cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
    print("After temporary update:", cursor.fetchone()[0], flush=True)

    # Rollback temporary changes
    conn.rollback()
    cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
    print("After rollback:", cursor.fetchone()[0], flush=True)

    # Permanent update (commit)
    cursor.execute(
        "UPDATE Paper SET title='Atomic Demo Committed' WHERE paper_id=1;")
    conn.commit()
    cursor.execute("SELECT title FROM Paper WHERE paper_id=1;")
    print("After commit:", cursor.fetchone()[0], flush=True)

except mysql.connector.Error as err:
    conn.rollback()
    print("Atomicity error:", err, flush=True)

# ------------------------------------------------------
# 2. Consistency Example
# ------------------------------------------------------
print("=== Consistency Example ===", flush=True)

try:
    # ---------------------------------------------
    # Step 0: Check Foreign Keys + Table Constraints
    # ---------------------------------------------
    print("\n-- Validating Constraints Before Insert --", flush=True)


    # Check table constraints (PK, FK, NOT NULL, UNIQUE)
    cursor.execute("SHOW CREATE TABLE Comment;")
    table_def = cursor.fetchone()[1]
    print("Table Constraints Verified (PK, FK, NOT NULL, UNIQUE)\n")

    # ---------------------------------------------
    # Step 1: Count before insert
    # ---------------------------------------------
    cursor.execute("SELECT COUNT(*) FROM Comment;")
    before = cursor.fetchone()[0]
    print("Before insert:", before, flush=True)

    # ---------------------------------------------
    # Step 2: Temporary insert (auto transaction)
    # ---------------------------------------------
    cursor.execute("""
        INSERT INTO Comment (paper_id, member_id, comment_text)
        SELECT 1, 2, 'Consistency Temp'
        WHERE NOT EXISTS (SELECT 1 FROM Comment WHERE comment_text='Consistency Temp');
    """)
    cursor.execute("SELECT COUNT(*) FROM Comment;")
    print("After temporary insert (before rollback):",
          cursor.fetchone()[0], flush=True)

    # ---------------------------------------------
    # Step 3: Rollback → should restore original count
    # ---------------------------------------------
    conn.rollback()
    cursor.execute("SELECT COUNT(*) FROM Comment;")
    print("After rollback:", cursor.fetchone()[0], flush=True)

    # ---------------------------------------------
    # Step 4: Permanent insert (commit)
    # ---------------------------------------------
    cursor.execute("""
        INSERT INTO Comment (paper_id, member_id, comment_text)
        SELECT 1, 2, 'Consistency Committed'
        WHERE NOT EXISTS (SELECT 1 FROM Comment WHERE comment_text='Consistency Committed');
    """)
    conn.commit()

    cursor.execute("SELECT COUNT(*) FROM Comment;")
    print("After commit:", cursor.fetchone()[0], "\n", flush=True)

except mysql.connector.Error as err:
    conn.rollback()
    print("Consistency error:", err, "\n", flush=True)


# ------------------------------------------------------
# 3. Isolation Example
# ------------------------------------------------------
print("=== Isolation Example (Rollback & Commit) ===", flush=True)

try:
    # Single connection for demonstration
    conn = mysql.connector.connect(
        host="localhost", user="root", password="Sneha@96", database="FINAL_URP")
    cursor = conn.cursor()

    # Set isolation level
    cursor.execute("SET SESSION TRANSACTION ISOLATION LEVEL READ COMMITTED;")

    # --- Temporary update (rollback) ---
    cursor.execute("SELECT score FROM Review WHERE review_id=1;")
    original_score = cursor.fetchone()[0]
    temp_score = original_score + 2
    print("Before temporary update:", original_score, flush=True)

    cursor.execute(
        "UPDATE Review SET score=%s WHERE review_id=1;", (temp_score,))
    cursor.execute("SELECT score FROM Review WHERE review_id=1;")
    print("After temporary update (before rollback):",
          cursor.fetchone()[0], flush=True)

    conn.rollback()  # undo temporary change
    cursor.execute("SELECT score FROM Review WHERE review_id=1;")
    print("After rollback:", cursor.fetchone()[0], flush=True)

    # --- Permanent update (commit) ---
    cursor.execute("SELECT score FROM Review WHERE review_id=1;")
    row = cursor.fetchone()
    score_for_commit = row[0] + 3
    print("Before permanent update:", row[0], flush=True)

    cursor.execute(
        "UPDATE Review SET score=%s WHERE review_id=1;", (score_for_commit,))
    conn.commit()  # make permanent
    cursor.execute("SELECT score FROM Review WHERE review_id=1;")
    print("After commit:", cursor.fetchone()[0], "\n", flush=True)

except mysql.connector.Error as err:
    conn.rollback()
    print("Isolation error:", err, "\n", flush=True)

finally:
    cursor.close()
    conn.close()


# ------------------------------------------------------
# 4. Durability Example
# ------------------------------------------------------
import time  # for unique timestamp

print("=== Durability Example ===", flush=True)
try:
    conn.start_transaction()

    # Original member count
    cursor.execute("SELECT COUNT(*) FROM Member;")
    original_count = cursor.fetchone()[0]
    print("Original Member count:", original_count, flush=True)

    # Temporary insert (rollback later)
    cursor.execute("""
        INSERT INTO Member (name, email, role, dept_id)
        VALUES ('Durable Temp', 'temp@uni.edu', 'Student', 1);
    """)
    cursor.execute("SELECT COUNT(*) FROM Member;")
    print("After temporary insert (before rollback):",
          cursor.fetchone()[0], flush=True)

    # Rollback temporary insert
    conn.rollback()
    cursor.execute("SELECT COUNT(*) FROM Member;")
    print("After rollback:", cursor.fetchone()[0], flush=True)

    # Permanent insert (with unique email)
    unique_email = f"commit_{int(time.time())}@uni.edu"  # ensures uniqueness
    cursor.execute(f"""
        INSERT INTO Member (name, email, role, dept_id)
        VALUES ('Durable Commit', '{unique_email}', 'Student', 1);
    """)
    conn.commit()
    cursor.execute("SELECT COUNT(*) FROM Member;")
    print("After commit:", cursor.fetchone()[0], "\n", flush=True)

except mysql.connector.Error as err:
    conn.rollback()
    print("Durability error:", err, "\n", flush=True)


# ----------------------------
# Close main connection
# ----------------------------
cursor.close()
conn.close()
print("Main connection closed.", flush=True)


ModuleNotFoundError: No module named 'mysql'